In [167]:
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import jieba
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from kafka import KafkaConsumer
from kafka import TopicPartition
import json
import time
import csv
import sched

In [168]:
def kafka_consumer(topic,offset):    
    news=[]
    consumer = KafkaConsumer(bootstrap_servers='192.168.99.100:9092')
    parti = TopicPartition(topic, 0)
    consumer.assign([parti])
    consumer.seek(parti,offset)
    for msg in consumer:
        print(msg.offset)
        print(msg.value.decode('utf-8'))
        if msg.value.decode('utf-8') == 'done':
            offset=msg.offset+1
            break
        news.append(msg.value.decode('utf-8'))
    return offset,news

In [169]:
def similar_analysis(f):
    corpus = []
    for line in f:
        corpus.append(" ".join(jieba.cut(line.split(',')[0], cut_all=False)))

    cv = CountVectorizer()
    term_doc = cv.fit_transform(corpus)

    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(corpus)
    words = vectorizer.get_feature_names()

    transformer = TfidfTransformer()
    tfidf = transformer.fit_transform(term_doc)
    fv = tfidf.toarray()

    #for i in range(len(fv)):
        #print('----Document %d----' % (i))
        #print(fv[i])

    sort_list = []
    for i in range(len(fv)):
        for j in range(i,len(fv)):
            if i != j:
                c_s = cosine_similarity([fv[i]], [fv[j]])
                #if c_s[0] != 0:
                #if i==0 or j ==0:
                #print('Doc ',i,' and ',j,' cosine similarity : ',c_s[0])
                sort_list.append([i,j,c_s[0]])
                sort_list.append([j,i,c_s[0]])
    sort_list = sorted(sort_list, reverse=False,key = lambda x : x[0])
    count=-1
    total=[]
    for i in range(len(sort_list)):
        if sort_list[i][0] != count:
            count=count+1
            total.append([sort_list[i]])
        else:
            total[count].append(sort_list[i])
    result=[]
    for i in range(len(fv)):
        total[i] = sorted(total[i], reverse=True,key = lambda x : x[2])
        temp=[str(i)]
        for j in range(5):
            #print(total[i][j])
            temp.append((str(total[i][j][1])+'('+str(total[i][j][2][0])+')'))
        result.append(temp)
    """for i in range(len(sort_list)):
        print(sort_list[i])
        firsts = []
        seconds = []
        for j in range(len(words)):
            if tfidf[sort_list[i][0],j] > 1e-5:
                firsts.append(words[j])
            if tfidf[sort_list[i][1],j] > 1e-5:
                seconds.append(words[j])
        print(sort_list[i][0],','.join(firsts))
        print(sort_list[i][1],','.join(seconds))"""
    #print(result)
    return result

In [170]:
def save_json(temp,filename):
    news=[]
    for i in range(len(temp)):
        news.append({
            'title': temp[i],
            'id': i
        }) 
    with open(filename, 'w', encoding='utf-8') as f:
            json.dump(news, f, indent=2, sort_keys=True, ensure_ascii=False)

In [171]:
def save_csv(result,filename):
    with open(filename, 'w', newline='') as csvfile:
        for i in range(len(result)):
            testwriter = csv.writer(csvfile)
            #print(result[i])
            testwriter.writerow(result[i])

In [172]:
def main(offset1,offset2,count):
    topic1='poli'
    topic2='ente'
    schedule = sched.scheduler ( time.time, time.sleep )
    
    offset1,news1=kafka_consumer(topic1,offset1)
    time.sleep(1)
    offset2,news2=kafka_consumer(topic2,offset2)
    time.sleep(1)
    save_json(news1,'poli_news'+str(count)+'.json')
    save_json(news2,'ente_news'+str(count)+'.json')
    result1=similar_analysis(news1)
    result2=similar_analysis(news2)
    save_csv(result1,'poli_result'+str(count)+'.csv')
    save_csv(result2,'ente_result'+str(count)+'.csv')
    
    count=count+1
    schedule.enter(10800,0,main,(offset1,offset2,count))  
    schedule.run()

In [173]:
offset1=0
offset2=0
count=1
main(offset1,offset2,count)

0
法總統：一旦決定動武 鎖定敘利亞政府化武能力
1
法總統：一旦決定動武 鎖定敘利亞政府化武能力
2
紐約匯市─歐洲央行量化寬鬆計畫(QE)即將結束 美元走低 歐元上漲
3
習近平承諾進一步開放中國經濟 川普：歡迎
4
敘傳化武攻擊 美英法領袖：世界須回應
5
公海狂追 印尼逮獲疑似奴隸船
6
決戰新北！侯友宜、蘇貞昌超級比一比
7
新任國家安全顧問就職一天後 川普的首席國土安全顧問辭職
8
新任國家安全顧問就職一天後 川普的首席國土安全顧問辭職
9
中國演習甫落幕  美軍航母打擊群巡弋南海
10
川普取消打丁美洲之行監督對敘出擊
11
敘利亞傳化武攻擊 安理會將表決美俄提案
12
美核子動力航母打擊群巡弋南海
13
中國演習甫落幕 美軍航母打擊群巡弋南海
14
匈牙利總理連任 矢言限制移民鞏固主權
15
敘利亞傳化武攻擊 安理會將表決美俄提案
16
玉山薩克斯風音樂會　溫馨慶玉管處33歲生日
17
桃市汽車商業公會參訪市議會　議長盼設汽車商城「集市」
18
樹林複合式災害演習 逼真
19
蔡總統授勳前部長馮世寬 表彰貢獻
20
智慧醫療趨勢 醫病關係更友善
21
朝外交官：階段、同步解決半島無核化
22
許昌關心基層訓練整備
23
張冠群視導國軍新竹醫院 勉永續服務
24
金正恩首度提及要與韓美會談
25
共軍建通訊干擾設備 「南海軍事化」
26
我敦睦艦隊抵尼加拉瓜 強化拉美邦交
27
張哲平視導臺東戰備工作
28
「杜魯門號」前進中東 接替「羅斯福號」
29
後備部召開演習任務提示
30
美揪敘化武元兇 不排除懲罰性攻擊
31
10軍團實施基層親考親教
32
陸軍聯合婚禮 開始報名
33
蔡總統：持續改善募兵制 強化國防自主
34
卡管逾90天 藍要彈劾潘文忠
35
桃、英合作打造智慧城市
36
藍議員參選人憂母雞無力 馬、郝、侯可望為火車頭帶動選情
37
川普是否拿台灣卡對戰中國？分析師：中美貿易談判恐更複雜
38
川普是否拿台灣卡對戰中國？分析師：中美貿易談判恐更複雜
39
匈牙利總理連任 矢言限制移民鞏固主權
40
將北上接府秘 陳菊高雄市政會議哽咽告別
41
被爆中國兼課 葉俊榮：僅短期講學非聘任
42
一張舊照片　讓蔡英文紅了眼眶
43
監督對敘出擊 川普取消訪問拉丁美洲
44
誰是侯友宜「傳話人」？葉元之賭請吃百份雞排
45
柯文哲施政報告 國民

121
《我們相遇的奇跡》 奪韓國週一週二劇收視冠軍
122
鋁箔紙變身亮晶晶金屬球？2種材料有趣小實驗
123
史上震度最強POPPING 阿嬤的機械舞狂到爆
124
20年鄰反目！半夜砸蛋、灑啤酒洩憤
125
耗時55天！「權力遊戲」拍完全劇最大戰役
126
韓國女歌手 CL社交網站發泳裝照秀火辣身材
127
韓瑜化身「越南阿阮」復仇　網友盛讚：演技大爆發
128
孫安佐「精神不濟」解套？　邱彰遭警長打臉
129
哈利梅根婚禮從簡 不請政治人物
130
NBA／老婆懷孕還亂搞　騎士大前鋒又被爆料
131
傳丁海寅續約 FNC娛樂否認
132
宋玧妸談老公人氣 稱薛景求收到無數粉絲信件
133
佼哥突襲！亞綸急掛電話【佼心食堂】
134
《我們遇見的奇蹟》收視下降 仍居月火劇冠軍
135
祖克柏證詞 華爾街買單 臉書股價大漲4.5% 2年來最棒的一天
136
祖克柏證詞 華爾街買單 臉書股價大漲4.5% 2年來最棒的一天
137
《噤界》女主角艾蜜莉布朗：幸好我沒當上黑寡婦
138
瞧不起漫威電影？柔伊莎達娜痛斥好萊塢菁英論者
139
SpeXial男男戀曝光　腐女暴動了
140
試鏡間》男版周子瑜拚演技 張豐豪拍霸凌戲瘦四公斤
141
鏡爆頭條》和雷婕熙同居ING 林柏叡勾王淨公園野戰
142
鏡爆頭條》在家昏倒送醫 大S41歲喜懷第三胎
143
布萊德彼特傳新戀情 女方是知名建築師MIT教授
144
準人妻孟耿如傳氣胸發作　半夜送急診
145
佛系正妹賣紙紮陰宅　清明連假賺到翻
146
側寫／K女郎大比拚！許維恩略勝一籌
147
愛拿亞裔男開玩笑　韓裔女藝人遭圍剿
148
楊丞琳玩直播催票　李榮浩上線湊一咖
149
有片／吳宗憲爆哭3分鐘 手抖寫1行字吐出道31年心酸
150
駁斥孫越病危 李明依「多年前已交代後事」
151
王柏傑自嘲「倒數第二個男朋友」卻發郭書瑤好友卡
152
何潤東曝出道新人苦 簽唱會只來3人
153
印尼富豪開生日趴 陳美鳳竟被當成飯局妹！
154
KlD招認曾偷吃 帶妹進許維恩香閨
155
一場深吻長達4分鐘 池昌旭啾啾聲成KISS匠人
156
瑞秋麥亞當斯低調產子 帶寶寶用餐被鄰桌發現才露餡
157
「唐朝豪放女」夏文汐復出 現在長這樣
158
聽證會上為自家辯護 祖克柏：臉書很安全 我和家人都用
159
臉書創辦人祖克柏出席國會聽

KeyboardInterrupt: 